<div style="background-image: url('images/chain.jpg'); background-size: cover; border-radius: 20px; width: 100%;">
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <center><h1 style="font-size: 50px; color: white; text-shadow: 0 0 5px black, 0 0 5px black;"><b>5. Final Workflow</b></h1></center>
    <center><h1 style="font-size: 30px; color: white;  text-shadow: 0 0 5px black, 0 0 5px black;">Computer Vision</h1></center>
    <br>
    <br>
    <br>
    <br>
    <br>
    <div style="color: white; margin-left:  5%; font-size: 20px;">
        <p>Realizado por: </p>
        <ul>
            <li>Beatriz Santos, nº 108593</li>
            <li>João Ferreira, nº 88639</li>
            <li>Rodrigo Sarroeira, nº 92761</li>
        </ul>
    </div>        
    <br>
    <br>
    <br><br></div>

In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras.applications import MobileNet

In [7]:
MODEL_FILE = "MobileNet/frozen_inference_graph.pb"
CONFIG_FILE = "MobileNet/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt"
CLASS_FILE = "MobileNet/object_detection_classes_coco.txt"
VIDEO_FILE = "videos/v1.mp4"
CONFIDENCE_THRESHOLD = 0.5

In [8]:
with open(CLASS_FILE, 'r') as f:
    class_names = f.read().split('\n')

COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

In [9]:
# Plane detector
SSDmodel = cv2.dnn.readNet(model=MODEL_FILE,
                           config=CONFIG_FILE,
                           framework="TensorFlow")

# Plane classifier
mobile_net = tf.keras.models.load_model("ClassNets\MobileNet.h5")

In [11]:
videoStream = cv2.VideoCapture(VIDEO_FILE)
width = int(videoStream.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(videoStream.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('videos/output1.mp4', fourcc, 20.0, (width, height))
classes = os.listdir("images/crop_new")

while videoStream.isOpened():
    ret, frame = videoStream.read()
    if not ret:
        break

    img_height, img_width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(image=frame, size=(300, 300), swapRB=True)
    SSDmodel.setInput(blob)
    output = SSDmodel.forward()

    for detection in output[0, 0, :, :]:
        if detection[2] > CONFIDENCE_THRESHOLD:
            class_id = detection[1]
            class_name = class_names[int(class_id) - 1]
            color = COLORS[int(class_id)]

            bbox_x = int(detection[3] * img_width)
            bbox_y = int(detection[4] * img_height)
            bbox_width = int(detection[5] * img_width)
            bbox_height = int(detection[6] * img_height)

            plane = frame[bbox_y:bbox_y+bbox_height, bbox_x:bbox_x+bbox_width]
            data = np.empty((1, 224, 224, 3))
            data[0] = cv2.resize(plane, (224, 224), interpolation=cv2.INTER_AREA)
            prediction = np.argmax(mobile_net.predict_on_batch(data))
            cv2.rectangle(frame, (bbox_x, bbox_y), (bbox_width, bbox_height), color, thickness=2)
            cv2.putText(frame, classes[prediction], (bbox_x, bbox_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            out.write(frame)

out.release()